<a href="https://colab.research.google.com/github/nceder/qpb4e/blob/main/code/Chapter%2024/Chapter_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 24 Saving Data

# 24.2 SQLite: Using the sqlite3 database

In [ ]:
import sqlite3
conn = sqlite3.connect("datafile.db")

In [ ]:
cursor = conn.cursor()
cursor

In [ ]:
cursor.execute("create table people (id integer primary key, name text, count integer)")
cursor.execute("insert into people (name, count) values ('Bob', 1)")
cursor.execute("insert into people (name, count) values (?, ?)",
               ("Jill", 15))
conn.commit()

In [ ]:
cursor.execute("insert into people (name, count) values (:username, :usercount)",
                  {"username": "Joe", "usercount": 10})

In [ ]:
result = cursor.execute("select * from people")
print(result.fetchall())

[(1, 'Bob', 1), (2, 'Jill', 15), (3, 'Joe', 10)]


In [ ]:
result = cursor.execute("select * from people where name like :name",
                        {"name": "bob"})
print(result.fetchall())

[(1, 'Bob', 1)]


In [ ]:
cursor.execute("update people set count=? where name=?", (20, "Jill"))
result = cursor.execute("select * from people")
print(result.fetchall())

[(1, 'Bob', 1), (2, 'Jill', 20), (3, 'Joe', 10)]


In [ ]:
result = cursor.execute("select * from people")
for row in result:
    print(row)


(1, 'Bob', 1)
(2, 'Jill', 20)
(3, 'Joe', 10)


In [ ]:
cursor.execute("update people set count=? where name=?", (20, "Jill"))
conn.commit()
conn.close()

# 24.4 Making database handling easier with an ORM

## 24.4.1 SQLAlchemy

In [ ]:
from sqlalchemy import create_engine, select, MetaData, Table, Column, Integer, String
from sqlalchemy.orm import sessionmaker

In [ ]:
dbPath = 'datafile2.db'
engine = create_engine('sqlite:///%s' % dbPath)
metadata = MetaData()
people  = Table('people', metadata,
                Column('id', Integer, primary_key=True),
                Column('name', String),
                Column('count', Integer),
               )
Session = sessionmaker(bind=engine)
session = Session()
metadata.create_all(engine)

In [ ]:
people_ins = people.insert().values(name='Bob', count=1)
str(people_ins)

'INSERT INTO people (name, count) VALUES (:name, :count)'

In [ ]:
session.execute(people_ins)

In [ ]:
session.commit()

In [ ]:
session.execute(people_ins, [
    {'name': 'Jill', 'count':15},
    {'name': 'Joe', 'count':10}
])

In [ ]:
session.commit()
people_query = select(people)
result = session.execute(people_query)
for row in result:
    print(row)


(1, 'Bob', 1)
(2, 'Jill', 15)
(3, 'Joe', 10)


In [ ]:
result = session.execute(select(people).where(people.c.name == 'Jill'))
for row in result:
    print(row)


(2, 'Jill', 15)


In [ ]:
result = session.execute(people.update().values(count=20).where (people.c.name == 'Jill'))
session.commit()
result = session.execute(select(people).where(people.c.name == 'Jill'))
for row in result:
    print(row)


(2, 'Jill', 20)


### Mapping table objects to classes

In [ ]:
from sqlalchemy.orm import declarative_base
Base = declarative_base()
class People(Base):
    __tablename__ = "people"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    count = Column(Integer)

results = session.query(People).filter_by(name='Jill')
for person in results:
    print(person.id, person.name, person.count)


2 Jill 20


In [ ]:
new_person = People(name='Jane', count=5)
session.add(new_person)
session.commit()
results = session.query(People).all()
for person in results:
    print(person.id, person.name, person.count)


1 Bob 1
2 Jill 20
3 Joe 10
4 Jane 5


In [ ]:
jill = session.query(People).filter_by(name='Jill').first()
jill.name

'Jill'

In [ ]:
jill.count = 22
session.add(jill)
session.commit()
results = session.query(People).all()
for person in results:
    print(person.id, person.name, person.count)


1 Bob 1
2 Jill 22
3 Joe 10
4 Jane 5


In [ ]:
jane = session.query(People).filter_by(name='Jane').first()
session.delete(jane)
session.commit()
jane = session.query(People).filter_by(name='Jane').first()
print(jane)

None


## 24.4.2 Using Alembic for database schema changes

In [ ]:
! rm -rf alembic/

In [ ]:
! pip install alembic
! alembic init alembic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.8 MB/s eta 0:00:00
Creating directory '/content/alembic' ...  done
Creating directory '/content/alembic/versions' ...  done
Generating /content/alembic/env.py ...  done
Generating /content/alembic/script.py.mako ...  done
Generating /content/alembic/README ...  done
Generating /content/alembic.ini ...  done
Please edit configuration/connection/logging settings in '/content/alembic.ini' before proceeding.


In [ ]:
# This cell will update the alembic.ini file

! sed -i 's/driver:\/\/user:pass@localhost\/dbname/sqlite:\/\/\/datafile.db/' alembic.ini

In [ ]:
# This cell create the first revision script
result = ! alembic revision -m "create an address table"
filename= result[0].replace('Generating ', "").replace(" ...  done","")
version = filename.split("/")[-1].split("_")[0]

'dbb13fdbd5ce'

In [ ]:
# This cell updates the revision script's upgrade() and downgrade() functions
upgrade_cmd = """def upgrade() -> None:
    op.create_table(
        'address',
        sa.Column('id', sa.Integer, primary_key=True),
        sa.Column('address', sa.String(50), nullable=False),
        sa.Column('city', sa.String(50), nullable=False),
        sa.Column('state', sa.String(20), nullable=False),
    )
"""
downgrade_cmd = """def downgrade() -> None:
    op.drop_table('address')"""

version_file = open(filename).read()
version_file = version_file.replace("""def upgrade() -> None:
    pass""", upgrade_cmd)
version_file = version_file.replace("""def downgrade() -> None:
    pass""", downgrade_cmd)
print(version_file)

open(filename, "w").write(version_file)

In [ ]:
for table in metadata.sorted_tables:
    print(table.name)

people


In [ ]:
! alembic upgrade head

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> dbb13fdbd5ce, create an address table


In [ ]:
metadata.sorted_tables

[Table('people', MetaData(), Column('id', Integer(), table=<people>, primary_key=True, nullable=False), Column('name', String(), table=<people>), Column('count', Integer(), table=<people>), schema=None)]

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()


In [ ]:
metadata.reflect(engine)
metadata.tables.keys()
session.commit()

In [ ]:
! alembic downgrade -1


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running downgrade dbb13fdbd5ce -> , create an address table


# 24.6 key:value stores with Redis

In [ ]:
!pip install redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 5.0 MB/s eta 0:00:00


In [ ]:
import redis

r = redis.Redis(
    # set your host here
    host='',
    port=10032,
    # set your password below
    password='')

#### Basic operations

In [ ]:
r.delete('words', 'counter', 'a_key')

2

In [ ]:
r.keys()

[]

#### Array operations

In [ ]:
r.set('a_key', 'my value')

True

In [ ]:
r.keys()

[b'a_key']

In [ ]:
v = r.get('a_key')
v

b'my value'

In [ ]:
r.incr('counter')

1

In [ ]:
r.get('counter')

b'1'

In [ ]:
r.incr('counter')

2

In [ ]:
r.get('counter')

b'2'

In [ ]:
r.rpush("words", "one")

1

In [ ]:
r.rpush("words", "two")

2

In [ ]:
r.lrange("words", 0, -1)

[b'one', b'two']

In [ ]:
r.rpush("words", "three")

3

In [ ]:
r.lrange("words", 0, -1)

[b'one', b'two', b'three']

In [ ]:
r.llen("words")

3

In [ ]:
r.lpush("words", "zero")

4

In [ ]:
r.lrange("words", 0, -1)

[b'zero', b'one', b'two', b'three']

In [ ]:
r.lrange("words", 2, 2)

[b'two']

In [ ]:
r.lindex("words", 1)

b'one'

In [ ]:
r.lindex("words", 2)

b'two'

### Expiration of values

In [ ]:
r.setex("timed", 10,  "10 seconds")

True

In [ ]:
r.pttl("timed")

4487

In [ ]:
r.pttl("timed")

-2

In [ ]:
b"timed" in r.keys()

False

# MongoDB Atalas

In [37]:
! pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 10.1 MB/s eta 0:00:00


In [40]:
from pymongo import MongoClient
client = MongoClient(host='*** connection string here ***')   #A

In [67]:
import datetime
a_document = {'name': 'Jane',
              'age': 34,
              'interests': ['Python', 'databases', 'statistics'],
              'date_added': datetime.datetime.now()
}
db = client.my_data     #A
collection = db.docs   #B
result = collection.find_one()  #C
db.list_collection_names()

[]

In [81]:
collection.insert_one(a_document)


InsertOneResult(ObjectId('66dfb43b90d33ea950f087e2'), acknowledged=True)

In [82]:
collection.find_one()    #A

{'_id': ObjectId('66dfb43b90d33ea950f087e2'),
 'name': 'Jane',
 'age': 34,
 'interests': ['Python', 'databases', 'statistics'],
 'date_added': datetime.datetime(2024, 9, 10, 2, 51, 38, 287000)}

In [83]:
from bson.objectid import ObjectId
collection.find_one({"_id":ObjectId('66dfb43b90d33ea950f087e2')})  #B

{'_id': ObjectId('66dfb43b90d33ea950f087e2'),
 'name': 'Jane',
 'age': 34,
 'interests': ['Python', 'databases', 'statistics'],
 'date_added': datetime.datetime(2024, 9, 10, 2, 51, 38, 287000)}

In [84]:
collection.update_one({"_id":ObjectId('66dfb43b90d33ea950f087e2')}, {"$set": {"name":"Ann"}})       #C

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000002b7'), 'opTime': {'ts': Timestamp(1725937089, 16), 't': 695}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1725937089, 16), 'signature': {'hash': b'\x85J\x00\xf7\x0c\x94\xc57-O\x80|\xa9Z\xea\xa6\x91\xe4\x18\xe4', 'keyId': 7368510527980437523}}, 'operationTime': Timestamp(1725937089, 16), 'updatedExisting': True}, acknowledged=True)

In [85]:
collection.find_one({"_id":ObjectId('66dfb43b90d33ea950f087e2')})

{'_id': ObjectId('66dfb43b90d33ea950f087e2'),
 'name': 'Ann',
 'age': 34,
 'interests': ['Python', 'databases', 'statistics'],
 'date_added': datetime.datetime(2024, 9, 10, 2, 51, 38, 287000)}

In [86]:
collection.replace_one({"_id":ObjectId('66dfb43b90d33ea950f087e2')}, {"name":"Maria"})                 #D

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000002b7'), 'opTime': {'ts': Timestamp(1725937094, 26), 't': 695}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1725937094, 26), 'signature': {'hash': b'l\x10\xc0(g\x03/&Z\t\xddq\ts\x8aQ/\x9f`\xac', 'keyId': 7368510527980437523}}, 'operationTime': Timestamp(1725937094, 26), 'updatedExisting': True}, acknowledged=True)

In [87]:
collection.find_one({"_id":ObjectId('66dfb43b90d33ea950f087e2')})

{'_id': ObjectId('66dfb43b90d33ea950f087e2'), 'name': 'Maria'}

In [88]:
collection.delete_one({"_id":ObjectId('66dfb43b90d33ea950f087e2')}) #E

DeleteResult({'n': 1, 'electionId': ObjectId('7fffffff00000000000002b7'), 'opTime': {'ts': Timestamp(1725937103, 28), 't': 695}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1725937103, 29), 'signature': {'hash': b'}\xcch\xcbK\xab\x01"^\x08\x0c\xc5\xf3\xfb\xb4\x9f\x05I\xcew', 'keyId': 7368510527980437523}}, 'operationTime': Timestamp(1725937103, 28)}, acknowledged=True)

In [89]:
collection.find_one()

In [90]:
db.list_collection_names()

['docs']

In [91]:
collection.drop()
db.list_collection_names()

[]